# 간단한 LLM 앱 구축
이 문서에서는 간단한 LLM 앱을 만드는 방법을 기술한다.
여기서 만드는 앱은 LLM을 활용하여 번역을 수행한다.

## 라이브러리 설치
아래 명령어를 통해 LangChain 및 LLM을 사용하기 위한 라이브러리를 설치한다.

In [1]:
!pip install langchain langchain-openai

## LangSmith 연동
흐름을 분석을 위해 아래 변수 설정으로 LangSmith와 연동한다.

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "Insert Your LangSmith API Key"

## LLM 정의
LLM을 사용하기 위해 라이브러리를 호출하고 인스턴스를 생성한다.

In [ ]:
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "Insert Your OpenAI API Key"
model = ChatOpenAI(model="gpt-4o")

단순히 모델을 호출하기 위해 메시지 목록을 `.invoke` 메서드에 전달한다.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

## 출력 Parser
모델의 응답은 `AIMessage`로 분류된다.

이는 응답 문자열과 응답에 대한 다른 메타데이터를 포함한다.

애플리케이션에서는 문자열 응답만을 다루고 싶기에 간단한 출력 파서를 사용하여 응답을 추출할 수 있다.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

출력 Parser는 단독으로 사용 가능하다.

언어 모델 호출 결과를 저장한 후 Parser에 전달하는 방식으로도 사용이 가능하다.

In [ ]:
result = model.invoke(messages)
parser.invoke(result)

또한 모델과 출력 Parser를 Chaining할 수 있다.

Chain은 언어 모델의 입력 타입을 받고 출력 Parser의 출력 타입을 반환 한다.

In [ ]:
chain = model | parser
chain.invoke(messages)

## 프롬프트 템플릿

프롬프트 템플릿은 LangChain에서 사용자의 입력 내용과 시스템 메시지 추가 혹은 입력 포맷팅 등의 변환을 돕기 위해 설계 된 개념이다.

원시 사용자의 입력을 받아 언어 모델에 전달할 준비가 된 프롬프트를 반환한다.

여기서 제시하는 프롬프트 템플릿은 두개의 사용자 변수를 받는다.

- `language` : 번역할 언어
- `text` : 번역할 텍스트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

이 프롬프트 템플릿의 입력은 딕셔너리이다.

프롬프트 템플릿이 단독으로 무엇을 하는지는 아래 코드를 통해 확인할 수 있다.

In [ ]:
result = prompt_template.invoke({"language": "Korean", "text": "hi"})
result

메시지에 대한 직접 접근은 아래 코드를 통해 가능하다.

In [ ]:
result.to_messages()

세 가지 구성 요소를 Chaining 하는 코드는 아래와 같다.

In [ ]:
chain = prompt_template | model | parser
chain.invoke({"language": "Korean", "text": "hi"})